<a href="https://colab.research.google.com/github/Raisler/DataScience_Portfolio/blob/master/M%C3%A9trica_de_Similaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similaridade entre textos 



In [ ]:
import numpy as np
from matplotlib_venn import venn2
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
gpt = pd.read_csv("results_gpt4.csv")
mistral = pd.read_csv("results_mistral_dolphin_2.csv")

In [ ]:
gpt.drop(columns=['category', 'prompt', 'reviews', 'review_qty', 'token_qty'], inplace=True)
mistral.drop(columns=['category', 'prompt', 'reviews', 'review_qty', 'token_qty'], inplace=True)

In [ ]:
review = pd.merge(mistral, gpt, on='product')

review.columns = ['Index', 'Product', 'Mistral', 'GPT']

review = review[review['Mistral'] != 'Resumo não encontrado.']

In [ ]:
def medidor_de_similaridade(text1, text2):
  to_vect = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))
  result = []
  text1 = [text1]
  text2 = [text2]
  for comentario1 in text1:
    for comentario2 in text2:
      x1, x2 = to_vect.fit_transform([comentario1,comentario2])
      t1, t2 = x1.toarray(), x2.toarray()

      min = np.amin([t1, t2], axis = 0)
      sum = np.sum(min)
      count = np.sum([t1, t2][0])
      to_mean = sum/count
      result.append(to_mean)
  return result

In [ ]:
mistrals = review['Mistral'].to_list()
gpts = review['GPT'].to_list()

for i in range(0, 3):
    print(medidor_de_similaridade(mistrals[i], gpts[i]))

In [ ]:
original_or_not1 = ['''
Customers appreciate the expansive watch face and easily legible numerals, coupled with the comfortable leather strap and convenient indiglo light feature. This timepiece proves especially suitable for seniors or those requiring larger watches for easy reading without the need for glasses. Nevertheless, a few users encountered problems with the band losing its color and fading within a mere two months of purchase, necessitating a replacement.''']
original_or_not2 = ['''
Customers love the large face and easy-to-read numerals, as well as the comfortable leather strap and indiglo light. The watch is perfect for seniors or individuals who need larger watches to read without glasses. However, some users experienced issues with the band losing color and fading within two months of purchase, requiring a replacement.''']

medidor_de_similaridade(original_or_not1, original_or_not2)

In [ ]:
import spacy.cli
from pathlib import Path

spacy.cli.download("en_core_web_sm")

nlp = spacy.load("en_core_web_sm")
texto = "There were 70 children there. Preliminary findings were reported in today's New England Journal of Medicine. Earnings growth took a back seat. A small building in the back. A clear majority of senators back the bill. Enable the country to buy back debt. I was twenty-one back then."
doc1 = nlp(original_or_not1[0])
doc2 = nlp(original_or_not2[0])


In [ ]:
clas1 = pd.DataFrame({'TOKEN': [], 'LEMA': [], 'POS': []})
clas2 = pd.DataFrame({'TOKEN': [], 'LEMA': [], 'POS': []})

In [ ]:
print("\n(TOKEN, LEMA,POS)")

for token in doc1:
    df_teste = pd.DataFrame({'TOKEN': [token.text], 'LEMA': [token.lemma_], 'POS': [token.pos_]})
    clas1 = pd.concat([clas1, df_teste])

for token in doc2:
    df_teste = pd.DataFrame({'TOKEN': [token.text], 'LEMA': [token.lemma_], 'POS': [token.pos_]})
    clas2 = pd.concat([clas2, df_teste])

In [ ]:
tokens1 = clas1[clas1['POS'] == 'ADJ']['TOKEN']
tokens2 = clas2[clas2['POS'] == 'ADJ']['TOKEN']

tokens_str1 = ''
tokens_str2 = ''

for token in tokens1:
    tokens_str1 = tokens_str1 + ' ' + token

for token in tokens2:
    tokens_str2 = tokens_str2 + ' ' + token

In [ ]:
vect1 = CountVectorizer(analyzer = 'word', ngram_range= (1, 2)) 
vect2 = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))

x1 = vect1.fit([tokens_str1])
x2 = vect2.fit([tokens_str2])

to_set1 = set(x1.vocabulary_)
to_set2 = set(x2.vocabulary_)

intersection = to_set1.intersection(to_set2)
  
venn2([to_set1, to_set2], set_labels = ('texto 1', 'texto 2'))